In [230]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import re
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pandas as pd
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import re

In [231]:
def extract(URL):
    client_id = "5356afb958c84e71a2c37c43e2a2cbf2" 
    client_secret = "83e531491e9c458ba658ac30c4c56bc0"

    #use the clint secret and id details
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    # the URI is split by ':' to get the username and playlist ID
    playlist_id = URL.split("/")[4].split("?")[0]
    playlist_tracks_data = sp.playlist_tracks(playlist_id)

    #lists that will be filled in with features
    playlist_tracks_id = []
    playlist_tracks_titles = []
    playlist_tracks_artists = []
    playlist_tracks_first_artists = []

    #go through the dictionary to extract the data
    for track in playlist_tracks_data['items']:
        playlist_tracks_id.append(track['track']['id'])
        playlist_tracks_titles.append(track['track']['name'])
        # adds a list of all artists involved in the song to the list of artists for the playlist
        artist_list = []
        for artist in track['track']['artists']:
            artist_list.append(artist['name'])
        playlist_tracks_artists.append(artist_list)
        playlist_tracks_first_artists.append(artist_list[0])

    #create a dataframe
    features = sp.audio_features(playlist_tracks_id)
    features_df = pd.DataFrame(data=features, columns=features[0].keys())
    features_df['title'] = playlist_tracks_titles
    features_df['first_artist'] = playlist_tracks_first_artists
    features_df['all_artists'] = playlist_tracks_artists
    features_df = features_df[['id', 'title', 'first_artist', 'all_artists',
                                'danceability', 'energy', 'key', 'loudness',
                                'mode', 'acousticness', 'instrumentalness',
                                'liveness', 'valence', 'tempo',
                                'duration_ms', 'time_signature']]
    
    return features_df

In [232]:
URL='https://open.spotify.com/playlist/37i9dQZF1DWT2oR9BciC32?si=510a6b4bf3d44685'

In [233]:
complete_feature_set=pd.read_csv(r'D:\PTIT\data science\khdl2022\recommendation_app\data1\complete_feature.csv')
df=extract(URL)
song_df=pd.read_csv(r"D:\PTIT\data science\khdl2022\recommendation_app\data1\allsong_data.csv")

In [234]:
complete_feature_set

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key,mode,duration_ms_x,id
0,0.182996,0.1626,0.168547,0.025156,0.006245,0.001401,0.00942,0.162325,0.114421,4,0,226863,0UaMYEvWZi0ZqiDOoHU3YI
1,0.156680,0.1676,0.178715,0.023701,0.005000,0.005025,0.04840,0.185170,0.130453,5,0,198800,6I9VzXrHxO9rA9A5euc8Ak
2,0.134413,0.1516,0.170210,0.043659,0.000478,0.000000,0.01196,0.140481,0.090525,2,0,235933,0WqIKmW4BTrj3eJFmnCKMv
3,0.180567,0.1428,0.171892,0.029314,0.040361,0.000047,0.01042,0.163727,0.092087,4,0,267266,1AWQoqb9bSvzTjaLralEkT
4,0.172672,0.1212,0.176541,0.014823,0.011265,0.000000,0.06260,0.131062,0.086421,0,1,227600,1lzr43nnXAijIGYnCT8M8H
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34245,0.135425,0.0456,0.152570,0.014345,0.159036,0.013065,0.01888,0.080561,0.075718,2,1,189183,3uCHI1gfOUL5j5swEh0TcH
34246,0.099798,0.1454,0.175155,0.045114,0.017530,0.000000,0.02580,0.057916,0.066813,1,1,263679,0P1oO2gREMYUCoOkzYAyFu
34247,0.142105,0.1048,0.157060,0.016486,0.066667,0.011116,0.05960,0.053106,0.127762,7,1,189213,2oM4BuruDnEvk59IvIXCwn
34248,0.103036,0.0572,0.144276,0.025572,0.080723,0.000002,0.02620,0.051904,0.110930,8,1,194720,4Ri5TTUgjM96tbQZd5Ua7V


In [235]:
df

,id,title,first_artist,all_artists,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,3wn8HJNjkY4wzTBy35ZvQ6,bao tiền một mớ bình yên?,14 Casper,"[14 Casper, Bon Nghiêm]",0.429,0.3640,7,-10.006,1,0.4550,0.000000,0.0907,0.0857,173.630,300000,3
1,5iZHnufFUgATzrpGgH1K0W,Ánh Sao Và Bầu Trời,T.R.I,[T.R.I],0.662,0.2050,5,-14.005,0,0.8760,0.003540,0.1090,0.2680,139.911,260571,4
2,2LhaQUFez282XCuTDSTDcV,Cô Ta,Vũ.,[Vũ.],0.398,0.3770,9,-7.564,1,0.7370,0.000160,0.1360,0.2150,145.909,214500,4
3,7MObIQOs5ESkbcihA9dcsz,Có Khi (feat. Low G),Madihu,"[Madihu, Low G]",0.649,0.3010,8,-9.700,0,0.8980,0.006610,0.0928,0.2690,74.999,259333,4
4,7eDrdQ7PK9Xm1CZCKWsgmH,Anh Nhớ Ra (feat. TRANG),Vũ.,"[Vũ., TRANG]",0.559,0.4650,10,-6.219,1,0.5240,0.000001,0.1200,0.3240,79.961,276500,4
5,73fFRs03s0gtjggfMW213L,Ngã Tư Không Đèn,TRANG,"[TRANG, Khoa Vũ]",0.848,0.4320,0,-10.262,0,0.7610,0.000025,0.1490,0.6560,120.000,218250,4
6,014DA3BdnmD3kI5pBogH7c,Cứ Chill Thôi,Chillies,"[Chillies, Suni Hạ Linh, Rhymastic]",0.781,0.6080,8,-8.452,1,0.2170,0.000036,0.1580,0.6090,101.017,270000,4
7,2EZtTS76x5M6hK96CLclPd,Liệu Giờ,2T,"[2T, Văn]",0.658,0.4510,6,-7.721,1,0.5310,0.000034,0.1220,0.4720,179.862,258668,4
8,1EmMFSLRVkOszCa4ul9z0F,3107 3,W/N,"[W/N, Duongg, Nau, titie]",0.663,0.3440,0,-14.025,1,0.9220,0.002980,0.0916,0.4690,135.904,240000,4
9,2WOpp4vm16unl0N3i3BNNv,Tình Đắng Như Ly Cà Phê,nân,"[nân, Ngơ]",0.754,0.0774,1,-20.617,1,0.9230,0.000000,0.0802,0.7010,116.071,215172,4


In [236]:
df['id'].values

array(['3wn8HJNjkY4wzTBy35ZvQ6', '5iZHnufFUgATzrpGgH1K0W',
       '2LhaQUFez282XCuTDSTDcV', '7MObIQOs5ESkbcihA9dcsz',
       '7eDrdQ7PK9Xm1CZCKWsgmH', '73fFRs03s0gtjggfMW213L',
       '014DA3BdnmD3kI5pBogH7c', '2EZtTS76x5M6hK96CLclPd',
       '1EmMFSLRVkOszCa4ul9z0F', '2WOpp4vm16unl0N3i3BNNv',
       '4AgFXIVT7hVhBZ5DEOaLu3', '5Bti0azlFhMattVY76qFr9',
       '6jcG3yZ0e2CSI6omkYO1ut', '0MscDl2UeYsharGGs8d9v9',
       '5uyNAX6MazVAjBjVeOsTvp', '4Iz5EWPCAT8jYNk4IpLoll',
       '7oBUoEbCDF748UkHnLusUB', '1CyYcBdukmFjFibDZaDlHc',
       '2FWhEGjP3wd6ZJKYfVpPvS', '6xyOEGUK1uKCSROp6RhkdS',
       '7wkX67ruxr3x2GFz6vyIX9', '7lZ07tAiDnmUE7cKw0jfSZ',
       '4CjJf5bzvv5ZuLf2FJqlWY', '2rGCYAabfA63PIzhQm7hVq',
       '0OYyivKo7eLtmHzYagW60S', '6GJi4nSZOiTOU2FhsgxWyH',
       '2PYDUM2IC01FOTDLQdFxJP', '27F22WcGDyl4Z2KylVJkXL',
       '5IO873C4IP8pXdsXcJiBes', '34VCllHNCrJopKy7uIL0pw',
       '696w9BHmEY62myVCSDyrYW', '3pvYWi5tsEn2pM9y3zaApN',
       '0AJi38NumMDXqo8pRw6zdh', '6JCgjW0daPZ2hkkciOPTX4

# gemerate_playlist_feature

In [237]:
complete_feature_set['id'].isin(df['id'].values)

0        False
1        False
2        False
3        False
4        False
         ...  
34245    False
34246    False
34247    False
34248    False
34249    False
Name: id, Length: 34250, dtype: bool

In [238]:
complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(df['id'].values)]

In [239]:
if complete_feature_set_playlist.empty==True:
    client_id = "5356afb958c84e71a2c37c43e2a2cbf2" 
    client_secret = "83e531491e9c458ba658ac30c4c56bc0"

    #use the clint secret and id details
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    playlist_id = URL.split("/")[4].split("?")[0]
    playlist_tracks_data = sp.playlist_tracks(playlist_id)

    playlist_tracks_id = []
    for track in playlist_tracks_data['items']:
        playlist_tracks_id.append(track['track']['id'])
    features = sp.audio_features(playlist_tracks_id)
    complete_feature_set_playlist = pd.DataFrame(data=features, columns=features[0].keys())
    complete_feature_set_playlist=complete_feature_set_playlist[['danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo','key','mode','duration_ms','id']]


In [240]:
complete_feature_set_playlist

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key,mode,duration_ms,id
0,0.429,0.3640,-10.006,0.0547,0.4550,0.000000,0.0907,0.0857,173.630,7,1,300000,3wn8HJNjkY4wzTBy35ZvQ6
1,0.662,0.2050,-14.005,0.0349,0.8760,0.003540,0.1090,0.2680,139.911,5,0,260571,5iZHnufFUgATzrpGgH1K0W
2,0.398,0.3770,-7.564,0.0270,0.7370,0.000160,0.1360,0.2150,145.909,9,1,214500,2LhaQUFez282XCuTDSTDcV
3,0.649,0.3010,-9.700,0.0541,0.8980,0.006610,0.0928,0.2690,74.999,8,0,259333,7MObIQOs5ESkbcihA9dcsz
4,0.559,0.4650,-6.219,0.0241,0.5240,0.000001,0.1200,0.3240,79.961,10,1,276500,7eDrdQ7PK9Xm1CZCKWsgmH
5,0.848,0.4320,-10.262,0.0385,0.7610,0.000025,0.1490,0.6560,120.000,0,0,218250,73fFRs03s0gtjggfMW213L
6,0.781,0.6080,-8.452,0.0408,0.2170,0.000036,0.1580,0.6090,101.017,8,1,270000,014DA3BdnmD3kI5pBogH7c
7,0.658,0.4510,-7.721,0.0410,0.5310,0.000034,0.1220,0.4720,179.862,6,1,258668,2EZtTS76x5M6hK96CLclPd
8,0.663,0.3440,-14.025,0.0495,0.9220,0.002980,0.0916,0.4690,135.904,0,1,240000,1EmMFSLRVkOszCa4ul9z0F
9,0.754,0.0774,-20.617,0.0578,0.9230,0.000000,0.0802,0.7010,116.071,1,1,215172,2WOpp4vm16unl0N3i3BNNv


In [241]:

# Find all non-playlist song features
complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(df['id'].values)]
complete_feature_set_playlist_final = complete_feature_set_playlist.drop(columns = "id")


In [242]:
complete_feature_set_nonplaylist

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key,mode,duration_ms_x,id
0,0.182996,0.1626,0.168547,0.025156,0.006245,0.001401,0.00942,0.162325,0.114421,4,0,226863,0UaMYEvWZi0ZqiDOoHU3YI
1,0.156680,0.1676,0.178715,0.023701,0.005000,0.005025,0.04840,0.185170,0.130453,5,0,198800,6I9VzXrHxO9rA9A5euc8Ak
2,0.134413,0.1516,0.170210,0.043659,0.000478,0.000000,0.01196,0.140481,0.090525,2,0,235933,0WqIKmW4BTrj3eJFmnCKMv
3,0.180567,0.1428,0.171892,0.029314,0.040361,0.000047,0.01042,0.163727,0.092087,4,0,267266,1AWQoqb9bSvzTjaLralEkT
4,0.172672,0.1212,0.176541,0.014823,0.011265,0.000000,0.06260,0.131062,0.086421,0,1,227600,1lzr43nnXAijIGYnCT8M8H
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34245,0.135425,0.0456,0.152570,0.014345,0.159036,0.013065,0.01888,0.080561,0.075718,2,1,189183,3uCHI1gfOUL5j5swEh0TcH
34246,0.099798,0.1454,0.175155,0.045114,0.017530,0.000000,0.02580,0.057916,0.066813,1,1,263679,0P1oO2gREMYUCoOkzYAyFu
34247,0.142105,0.1048,0.157060,0.016486,0.066667,0.011116,0.05960,0.053106,0.127762,7,1,189213,2oM4BuruDnEvk59IvIXCwn
34248,0.103036,0.0572,0.144276,0.025572,0.080723,0.000002,0.02620,0.051904,0.110930,8,1,194720,4Ri5TTUgjM96tbQZd5Ua7V


In [243]:
complete_feature_set_playlist_final

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key,mode,duration_ms
0,0.429,0.3640,-10.006,0.0547,0.4550,0.000000,0.0907,0.0857,173.630,7,1,300000
1,0.662,0.2050,-14.005,0.0349,0.8760,0.003540,0.1090,0.2680,139.911,5,0,260571
2,0.398,0.3770,-7.564,0.0270,0.7370,0.000160,0.1360,0.2150,145.909,9,1,214500
3,0.649,0.3010,-9.700,0.0541,0.8980,0.006610,0.0928,0.2690,74.999,8,0,259333
4,0.559,0.4650,-6.219,0.0241,0.5240,0.000001,0.1200,0.3240,79.961,10,1,276500
5,0.848,0.4320,-10.262,0.0385,0.7610,0.000025,0.1490,0.6560,120.000,0,0,218250
6,0.781,0.6080,-8.452,0.0408,0.2170,0.000036,0.1580,0.6090,101.017,8,1,270000
7,0.658,0.4510,-7.721,0.0410,0.5310,0.000034,0.1220,0.4720,179.862,6,1,258668
8,0.663,0.3440,-14.025,0.0495,0.9220,0.002980,0.0916,0.4690,135.904,0,1,240000
9,0.754,0.0774,-20.617,0.0578,0.9230,0.000000,0.0802,0.7010,116.071,1,1,215172


In [244]:
if complete_feature_set_playlist_final.empty==False: print(1)

1


In [245]:
complete_feature_set_playlist_vector=complete_feature_set_playlist_final.sum(axis=0)

In [246]:
complete_feature_set_playlist_vector

danceability        3.192100e+01
energy              1.770840e+01
loudness           -5.418920e+02
speechiness         2.905000e+00
acousticness        3.516730e+01
instrumentalness    7.943920e-02
liveness            7.177900e+00
valence             2.028570e+01
tempo               6.074974e+03
key                 2.350000e+02
mode                4.200000e+01
duration_ms         1.212101e+07
dtype: float64

In [247]:
complete_feature_set_nonplaylist

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key,mode,duration_ms_x,id
0,0.182996,0.1626,0.168547,0.025156,0.006245,0.001401,0.00942,0.162325,0.114421,4,0,226863,0UaMYEvWZi0ZqiDOoHU3YI
1,0.156680,0.1676,0.178715,0.023701,0.005000,0.005025,0.04840,0.185170,0.130453,5,0,198800,6I9VzXrHxO9rA9A5euc8Ak
2,0.134413,0.1516,0.170210,0.043659,0.000478,0.000000,0.01196,0.140481,0.090525,2,0,235933,0WqIKmW4BTrj3eJFmnCKMv
3,0.180567,0.1428,0.171892,0.029314,0.040361,0.000047,0.01042,0.163727,0.092087,4,0,267266,1AWQoqb9bSvzTjaLralEkT
4,0.172672,0.1212,0.176541,0.014823,0.011265,0.000000,0.06260,0.131062,0.086421,0,1,227600,1lzr43nnXAijIGYnCT8M8H
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34245,0.135425,0.0456,0.152570,0.014345,0.159036,0.013065,0.01888,0.080561,0.075718,2,1,189183,3uCHI1gfOUL5j5swEh0TcH
34246,0.099798,0.1454,0.175155,0.045114,0.017530,0.000000,0.02580,0.057916,0.066813,1,1,263679,0P1oO2gREMYUCoOkzYAyFu
34247,0.142105,0.1048,0.157060,0.016486,0.066667,0.011116,0.05960,0.053106,0.127762,7,1,189213,2oM4BuruDnEvk59IvIXCwn
34248,0.103036,0.0572,0.144276,0.025572,0.080723,0.000002,0.02620,0.051904,0.110930,8,1,194720,4Ri5TTUgjM96tbQZd5Ua7V


In [248]:
complete_feature_set_playlist_final

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key,mode,duration_ms
0,0.429,0.3640,-10.006,0.0547,0.4550,0.000000,0.0907,0.0857,173.630,7,1,300000
1,0.662,0.2050,-14.005,0.0349,0.8760,0.003540,0.1090,0.2680,139.911,5,0,260571
2,0.398,0.3770,-7.564,0.0270,0.7370,0.000160,0.1360,0.2150,145.909,9,1,214500
3,0.649,0.3010,-9.700,0.0541,0.8980,0.006610,0.0928,0.2690,74.999,8,0,259333
4,0.559,0.4650,-6.219,0.0241,0.5240,0.000001,0.1200,0.3240,79.961,10,1,276500
5,0.848,0.4320,-10.262,0.0385,0.7610,0.000025,0.1490,0.6560,120.000,0,0,218250
6,0.781,0.6080,-8.452,0.0408,0.2170,0.000036,0.1580,0.6090,101.017,8,1,270000
7,0.658,0.4510,-7.721,0.0410,0.5310,0.000034,0.1220,0.4720,179.862,6,1,258668
8,0.663,0.3440,-14.025,0.0495,0.9220,0.002980,0.0916,0.4690,135.904,0,1,240000
9,0.754,0.0774,-20.617,0.0578,0.9230,0.000000,0.0802,0.7010,116.071,1,1,215172


# Generate_playlist_recos

In [249]:
non_playlist_df = song_df[song_df['id'].isin(complete_feature_set_nonplaylist['id'].values)]

In [250]:
non_playlist_df

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,name,danceability,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms_y,time_signature,artists_song
0,0,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks,0.904,...,0.810,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,Missy ElliottLose Control (feat. Ciara & Fat M...
1,1,Britney Spears,6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks,0.774,...,0.924,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4,Britney SpearsToxic
2,2,Beyoncé,0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0.664,...,0.701,99.259,audio_features,0WqIKmW4BTrj3eJFmnCKMv,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933,4,BeyoncéCrazy In Love
3,3,Justin Timberlake,1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,Throwbacks,0.892,...,0.817,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,spotify:track:1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267,4,Justin TimberlakeRock Your Body
4,4,Shaggy,1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,Throwbacks,0.853,...,0.654,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,spotify:track:1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600,4,ShaggyIt Wasn't Me
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34245,37,Jon D,3uCHI1gfOUL5j5swEh0TcH,spotify:artist:5HCypjplgh5uQezvBpOfXN,I Don't Know,spotify:album:2KEQtuVl1cYsTYtVRUrNVi,189183,Roots,thinking of you,0.669,...,0.402,83.024,audio_features,3uCHI1gfOUL5j5swEh0TcH,spotify:track:3uCHI1gfOUL5j5swEh0TcH,https://api.spotify.com/v1/tracks/3uCHI1gfOUL5...,https://api.spotify.com/v1/audio-analysis/3uCH...,189184,4,Jon DI Don't Know
34246,40,Big Words,0P1oO2gREMYUCoOkzYAyFu,spotify:artist:0sHN89qak07mnug3LVVjzP,The Answer,spotify:album:5jrsRHRAmetu5e7RRBoxj7,263679,"Hollywood, a Beautiful Coincidence",thinking of you,0.493,...,0.289,73.259,audio_features,0P1oO2gREMYUCoOkzYAyFu,spotify:track:0P1oO2gREMYUCoOkzYAyFu,https://api.spotify.com/v1/tracks/0P1oO2gREMYU...,https://api.spotify.com/v1/audio-analysis/0P1o...,263680,4,Big WordsThe Answer
34247,41,Allan Rayman,2oM4BuruDnEvk59IvIXCwn,spotify:artist:6Yv6OBXD6ZQakEljaGaDAk,25.22,spotify:album:3CbNgBzI7r9o0F6VjH9sTY,189213,Roadhouse 01,thinking of you,0.702,...,0.265,140.089,audio_features,2oM4BuruDnEvk59IvIXCwn,spotify:track:2oM4BuruDnEvk59IvIXCwn,https://api.spotify.com/v1/tracks/2oM4BuruDnEv...,https://api.spotify.com/v1/audio-analysis/2oM4...,189213,4,Allan Rayman25.22
34248,42,Jon Jason,4Ri5TTUgjM96tbQZd5Ua7V,spotify:artist:77bNdkKYBBmc30CisCA6tE,Good Feeling,spotify:album:2dZ7oVNQBeLlpoUYfbEsJP,194720,Good Feeling,thinking of you,0.509,...,0.259,121.633,audio_features,4Ri5TTUgjM96tbQZd5Ua7V,spotify:track:4Ri5TTUgjM96tbQZd5Ua7V,https://api.spotify.com/v1/tracks/4Ri5TTUgjM96...,https://api.spotify.com/v1/audio-analysis/4Ri5...,194720,4,Jon JasonGood Feeling


In [251]:
non_playlist_df['sim'] = cosine_similarity(complete_feature_set_nonplaylist.drop('id', axis = 1).values, complete_feature_set_playlist_vector.values.reshape(1, -1))[:,0]

In [252]:
complete_feature_set_nonplaylist.drop('id', axis = 1).values

array([[1.82995951e-01, 1.62600000e-01, 1.68546665e-01, ...,
        4.00000000e+00, 0.00000000e+00, 2.26863000e+05],
       [1.56680162e-01, 1.67600000e-01, 1.78714591e-01, ...,
        5.00000000e+00, 0.00000000e+00, 1.98800000e+05],
       [1.34412955e-01, 1.51600000e-01, 1.70209986e-01, ...,
        2.00000000e+00, 0.00000000e+00, 2.35933000e+05],
       ...,
       [1.42105263e-01, 1.04800000e-01, 1.57059555e-01, ...,
        7.00000000e+00, 1.00000000e+00, 1.89213000e+05],
       [1.03036437e-01, 5.72000000e-02, 1.44275563e-01, ...,
        8.00000000e+00, 1.00000000e+00, 1.94720000e+05],
       [1.29352227e-01, 9.22000000e-02, 1.64866329e-01, ...,
        6.00000000e+00, 0.00000000e+00, 2.57194000e+05]])

In [253]:
complete_feature_set_playlist_vector.values.reshape(1, -1)

array([[ 3.1921000e+01,  1.7708400e+01, -5.4189200e+02,  2.9050000e+00,
         3.5167300e+01,  7.9439200e-02,  7.1779000e+00,  2.0285700e+01,
         6.0749740e+03,  2.3500000e+02,  4.2000000e+01,  1.2121009e+07]])

In [254]:
non_playlist_df

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,name,danceability,...,tempo,type,id,uri,track_href,analysis_url,duration_ms_y,time_signature,artists_song,sim
0,0,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks,0.904,...,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,Missy ElliottLose Control (feat. Ciara & Fat M...,1.0
1,1,Britney Spears,6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks,0.774,...,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4,Britney SpearsToxic,1.0
2,2,Beyoncé,0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,0.664,...,99.259,audio_features,0WqIKmW4BTrj3eJFmnCKMv,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933,4,BeyoncéCrazy In Love,1.0
3,3,Justin Timberlake,1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,Throwbacks,0.892,...,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,spotify:track:1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267,4,Justin TimberlakeRock Your Body,1.0
4,4,Shaggy,1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,Throwbacks,0.853,...,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,spotify:track:1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600,4,ShaggyIt Wasn't Me,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34245,37,Jon D,3uCHI1gfOUL5j5swEh0TcH,spotify:artist:5HCypjplgh5uQezvBpOfXN,I Don't Know,spotify:album:2KEQtuVl1cYsTYtVRUrNVi,189183,Roots,thinking of you,0.669,...,83.024,audio_features,3uCHI1gfOUL5j5swEh0TcH,spotify:track:3uCHI1gfOUL5j5swEh0TcH,https://api.spotify.com/v1/tracks/3uCHI1gfOUL5...,https://api.spotify.com/v1/audio-analysis/3uCH...,189184,4,Jon DI Don't Know,1.0
34246,40,Big Words,0P1oO2gREMYUCoOkzYAyFu,spotify:artist:0sHN89qak07mnug3LVVjzP,The Answer,spotify:album:5jrsRHRAmetu5e7RRBoxj7,263679,"Hollywood, a Beautiful Coincidence",thinking of you,0.493,...,73.259,audio_features,0P1oO2gREMYUCoOkzYAyFu,spotify:track:0P1oO2gREMYUCoOkzYAyFu,https://api.spotify.com/v1/tracks/0P1oO2gREMYU...,https://api.spotify.com/v1/audio-analysis/0P1o...,263680,4,Big WordsThe Answer,1.0
34247,41,Allan Rayman,2oM4BuruDnEvk59IvIXCwn,spotify:artist:6Yv6OBXD6ZQakEljaGaDAk,25.22,spotify:album:3CbNgBzI7r9o0F6VjH9sTY,189213,Roadhouse 01,thinking of you,0.702,...,140.089,audio_features,2oM4BuruDnEvk59IvIXCwn,spotify:track:2oM4BuruDnEvk59IvIXCwn,https://api.spotify.com/v1/tracks/2oM4BuruDnEv...,https://api.spotify.com/v1/audio-analysis/2oM4...,189213,4,Allan Rayman25.22,1.0
34248,42,Jon Jason,4Ri5TTUgjM96tbQZd5Ua7V,spotify:artist:77bNdkKYBBmc30CisCA6tE,Good Feeling,spotify:album:2dZ7oVNQBeLlpoUYfbEsJP,194720,Good Feeling,thinking of you,0.509,...,121.633,audio_features,4Ri5TTUgjM96tbQZd5Ua7V,spotify:track:4Ri5TTUgjM96tbQZd5Ua7V,https://api.spotify.com/v1/tracks/4Ri5TTUgjM96...,https://api.spotify.com/v1/audio-analysis/4Ri5...,194720,4,Jon JasonGood Feeling,1.0


In [255]:
non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(50)

In [256]:
non_playlist_df_top_40

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,name,danceability,...,tempo,type,id,uri,track_href,analysis_url,duration_ms_y,time_signature,artists_song,sim
6218,83,Da Vinci's Notebook,3Zgr792YSKOtnpL1Zw7cqW,spotify:artist:4rIDY3ojaWd7Z9ky9SFc1g,Uncle Buford #1,spotify:album:4TP4fxDJUhtN1DrQRAnG7C,32146,Brontosaurus,All the songs,0.706,...,150.330,audio_features,3Zgr792YSKOtnpL1Zw7cqW,spotify:track:3Zgr792YSKOtnpL1Zw7cqW,https://api.spotify.com/v1/tracks/3Zgr792YSKOt...,https://api.spotify.com/v1/audio-analysis/3Zgr...,32147,3,Da Vinci's NotebookUncle Buford #1,1.0
19311,10,the bird and the bee,0kffMnc6Ox5dgTha5i60oK,spotify:artist:65XA3lk0aG9XejO8y37jjD,Fanfare,spotify:album:4ETVvsILHTLUHGYe3SSpgu,28680,Ray Guns Are Not Just The Future,Homework,0.618,...,141.935,audio_features,0kffMnc6Ox5dgTha5i60oK,spotify:track:0kffMnc6Ox5dgTha5i60oK,https://api.spotify.com/v1/tracks/0kffMnc6Ox5d...,https://api.spotify.com/v1/audio-analysis/0kff...,28680,4,the bird and the beeFanfare,1.0
10415,53,!llmind,3MZ0Obyv6AnO1anmGBJydv,spotify:artist:0bw7sLDsNQTScMpmsNFukJ,Take A Break - Interlude,spotify:album:5AgsHUKFxr5DApRCmulIqJ,48386,The Hamilton Mixtape,Musicals,0.805,...,161.707,audio_features,3MZ0Obyv6AnO1anmGBJydv,spotify:track:3MZ0Obyv6AnO1anmGBJydv,https://api.spotify.com/v1/tracks/3MZ0Obyv6AnO...,https://api.spotify.com/v1/audio-analysis/3MZ0...,48387,4,!llmindTake A Break - Interlude,1.0
9039,15,Relient K,4yyJjvy4rPSUKalm0M3ajG,spotify:artist:3nJWBJvK7uGvfp4iZh9CkN,Mmhmm,spotify:album:0k8YMJPGBjmiQF2rVHSQpR,18388,Mmhmm10,Music,0.390,...,129.628,audio_features,4yyJjvy4rPSUKalm0M3ajG,spotify:track:4yyJjvy4rPSUKalm0M3ajG,https://api.spotify.com/v1/tracks/4yyJjvy4rPSU...,https://api.spotify.com/v1/audio-analysis/4yyJ...,18388,4,Relient KMmhmm,1.0
1466,50,Parquet Courts,4Fg7pilwwzlTQtLXhO2ZlN,spotify:artist:23NIwARd4vPbxt3wwNnJ6k,Disney P.T.,spotify:album:1aYXn1WnQWLTmOpQtKVeYJ,72840,Light Up Gold,Garage Rock,0.175,...,199.701,audio_features,4Fg7pilwwzlTQtLXhO2ZlN,spotify:track:4Fg7pilwwzlTQtLXhO2ZlN,https://api.spotify.com/v1/tracks/4Fg7pilwwzlT...,https://api.spotify.com/v1/audio-analysis/4Fg7...,72840,4,Parquet CourtsDisney P.T.,1.0
27075,52,The U.S. Navy,53ndHj5sTIlIinRJ8VvyPn,spotify:artist:5cD2KAZODu6ocU2QWvbKJG,Saw an Old Lady,spotify:album:083DcSJuBVHbXEIaDiE89k,47240,"Workout to the Running Cadences U.S. Navy, Vol. 1",CARDIO,0.629,...,166.410,audio_features,53ndHj5sTIlIinRJ8VvyPn,spotify:track:53ndHj5sTIlIinRJ8VvyPn,https://api.spotify.com/v1/tracks/53ndHj5sTIlI...,https://api.spotify.com/v1/audio-analysis/53nd...,47240,4,The U.S. NavySaw an Old Lady,1.0
6935,8,Winds of Plague,7kLiH2qrpR99Maw3fXvwY8,spotify:artist:080xIP1qfuknBNYEYPuH1v,A Cold Day In Hell,spotify:album:49WqfwK15hYplA2BGXwFHq,74213,Decimate the Weak,Killin' It,0.289,...,194.999,audio_features,7kLiH2qrpR99Maw3fXvwY8,spotify:track:7kLiH2qrpR99Maw3fXvwY8,https://api.spotify.com/v1/tracks/7kLiH2qrpR99...,https://api.spotify.com/v1/audio-analysis/7kLi...,74213,3,Winds of PlagueA Cold Day In Hell,1.0
29016,92,Boards of Canada,3NFvPiE8u70nJymhP17Orx,spotify:artist:2VAvhf61GgLYmC6C8anyX1,Ataronchronon,spotify:album:0wBiN0fKhy0ywx732SpgUs,74133,The Campfire Headphase,work music,0.178,...,178.600,audio_features,3NFvPiE8u70nJymhP17Orx,spotify:track:3NFvPiE8u70nJymhP17Orx,https://api.spotify.com/v1/tracks/3NFvPiE8u70n...,https://api.spotify.com/v1/audio-analysis/3NFv...,74133,4,Boards of CanadaAtaronchronon,1.0
19647,65,Sufjan Stevens,0PXH84vcGY9no7JP9XiUG8,spotify:artist:4MXUO7sVCaFgFjoTI5ox5c,Ding! Dong!,spotify:album:6ZCbYO3B5eslkY3zHdss4A,55853,Songs For Christmas,It's Christmas Time!,0.504,...,151.278,audio_features,0PXH84vcGY9no7JP9XiUG8,spotify:track:0PXH84vcGY9no7JP9XiUG8,https://api.spotify.com/v1/tracks/0PXH84vcGY9n...,https://api.spotify.com/v1/audio-analysis/0PXH...,55853,3,Sufjan StevensDing! Dong!,1.0
29177,13,Stephen Schwartz,01GAyirO9mciPH3YV4JWNZ,spotify:artist:7daPb40K64rRtF36wd3ZOk,"I'm Not That Girl (Reprise) - From ""Wic

In [257]:
non_playlist_df_top_40=non_playlist_df_top_40[non_playlist_df_top_40.pos>90]

In [258]:
non_playlist_df_top_40

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,name,danceability,...,tempo,type,id,uri,track_href,analysis_url,duration_ms_y,time_signature,artists_song,sim
29016,92,Boards of Canada,3NFvPiE8u70nJymhP17Orx,spotify:artist:2VAvhf61GgLYmC6C8anyX1,Ataronchronon,spotify:album:0wBiN0fKhy0ywx732SpgUs,74133,The Campfire Headphase,work music,0.178,...,178.600,audio_features,3NFvPiE8u70nJymhP17Orx,spotify:track:3NFvPiE8u70nJymhP17Orx,https://api.spotify.com/v1/tracks/3NFvPiE8u70n...,https://api.spotify.com/v1/audio-analysis/3NFv...,74133,4,Boards of CanadaAtaronchronon,1.0
19260,97,Misfits,5Aox2oSTTA5jiQjtZZbMZe,spotify:artist:1cXi8ALPQCBHZbf0EgP4Ey,She,spotify:album:65KQFqWUDiwmVc9BjchWZy,82333,Legacy Of Brutality,Punk,0.145,...,187.518,audio_features,5Aox2oSTTA5jiQjtZZbMZe,spotify:track:5Aox2oSTTA5jiQjtZZbMZe,https://api.spotify.com/v1/tracks/5Aox2oSTTA5j...,https://api.spotify.com/v1/audio-analysis/5Aox...,82333,4,MisfitsShe,1.0
6877,176,Kanye West,3f42nSmfO2092cODYYKIrU,spotify:artist:5K4W6rqBFWDnAN6FQUkS6x,Workout Plan,spotify:album:1NRRN5RWwfuLmQdjshz0L7,46173,The College Dropout,its lit,0.715,...,115.114,audio_features,3f42nSmfO2092cODYYKIrU,spotify:track:3f42nSmfO2092cODYYKIrU,https://api.spotify.com/v1/tracks/3f42nSmfO209...,https://api.spotify.com/v1/audio-analysis/3f42...,46173,4,Kanye WestWorkout Plan,1.0
1515,99,Titus Andronicus,3YvChf5VPbuK6LAWQbybHH,spotify:artist:5lJ4XQ2hlPlxACN7q3xKL1,Food Fight!,spotify:album:4dx4Khp1YnIcZrTTdRHy1I,69240,Local Business,Garage Rock,0.302,...,178.623,audio_features,3YvChf5VPbuK6LAWQbybHH,spotify:track:3YvChf5VPbuK6LAWQbybHH,https://api.spotify.com/v1/tracks/3YvChf5VPbuK...,https://api.spotify.com/v1/audio-analysis/3YvC...,69240,4,Titus AndronicusFood Fight!,1.0
18381,198,Half Moon Run,62XpttqUaPz48pX2rGUzhi,spotify:artist:3ceQN2NVlLg1hgTzljDE4n,Throes,spotify:album:1s6FlVyzq929eQ3o1kw23L,54333,Sun Leads Me On,chill out,0.354,...,126.749,audio_features,62XpttqUaPz48pX2rGUzhi,spotify:track:62XpttqUaPz48pX2rGUzhi,https://api.spotify.com/v1/tracks/62XpttqUaPz4...,https://api.spotify.com/v1/audio-analysis/62Xp...,54333,3,Half Moon RunThroes,1.0
5091,231,William Fitzsimmons,5PduiXDYYKKWVsvgaZ4qcd,spotify:artist:41FEVJCBGidsJwbjq0KfgM,Please Please Please Let Me Get What I Want,spotify:album:2ySVkM6LaRl6bSdgVzFQ0u,113160,"Please, Please, Please: A Tribute to The Smiths",2012,0.424,...,177.711,audio_features,5PduiXDYYKKWVsvgaZ4qcd,spotify:track:5PduiXDYYKKWVsvgaZ4qcd,https://api.spotify.com/v1/tracks/5PduiXDYYKKW...,https://api.spotify.com/v1/audio-analysis/5Pdu...,113160,3,William FitzsimmonsPlease Please Please Let Me...,1.0
